In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

Reading the raw dataset.

In [ ]:
df = pd.read_csv(open("../raw/gpt4_political_bias.csv"), encoding="utf-8")
df

Renaming columns.

In [ ]:
df = df.rename(columns={"text": "body", "label": "leaning"})

Categorizing the leaning column.

In [ ]:
df["leaning"].unique()

In [ ]:
df["leaning"] = df["leaning"].replace({0: "right", 1: "right", 2: "center", 3: "left", 4: "left"})
df["leaning"] = df["leaning"].astype("category")

Dropping rows with missing data.

In [ ]:
df = df.dropna()

Printing duplicates.

In [ ]:
df[df["body"].duplicated(keep=False)]

Dropping the duplicates.

In [ ]:
df = df.drop_duplicates(subset="body")

Inspecting body length.

In [ ]:
df["body_length"] = df["body"].str.len()
df["body_word_count"] = df["body"].str.split().str.len()
df = df.sort_values(by="body_length")
df.head()

In [ ]:
# Ensuring to include the last (longest) item.
df["body_length"].plot.bar().xaxis.set_ticks([])

In [ ]:
df["body_length"].mean()

The leaning distribution.

In [ ]:
df.groupby("leaning", observed=True).size().plot.pie(autopct="%1.1f%%");

The distribution of body length sums per leaning.

In [ ]:
df.groupby("leaning", observed=True)["body_length"].sum().plot.pie(autopct="%1.1f%%", ylabel="");

Body length distribution by leaning.

In [ ]:
plt.figure(figsize=(10, 6))

for leaning in df["leaning"].unique():
    plt.plot(df[df["leaning"] == leaning].reset_index(drop=True)["body_length"], label=leaning)

plt.xlabel("downsampled index")
plt.ylabel("body length")
plt.title("body length by political leaning")
plt.legend(title="leaning")
plt.show()

In [ ]:
df.to_parquet("../preprocessed/gpt4_political_bias.parquet")